In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import joblib
sns.set()

In [ ]:
df = pd.read_csv('/kaggle/input/used-car-dataset/cardekho_imputated.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.count()

In [ ]:
df['Present_Price'] = (df['max_cost_price']+df['min_cost_price'])/2

In [ ]:
df.head()

In [ ]:
df.drop(['min_cost_price', 'max_cost_price'], axis = 1,inplace = True)

In [ ]:
df.drop(['Unnamed: 0', 'car_name','brand', 'model'], axis = 1,inplace = True)

In [ ]:
df.drop(['max_power', 'mileage'], axis = 1,inplace = True)

In [ ]:
df.drop(['engine'], axis = 1,inplace = True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['selling_price'] = df['selling_price'].astype('int64')
df['Present_Price'] = df['Present_Price'].astype('int64') 

In [ ]:
df.head()

In [ ]:
df.describe().astype('int64') 

In [ ]:
df.drop(df[df['Present_Price'] > 10000000].index, inplace = True)

In [ ]:
df.max()

In [ ]:
df.min()

In [ ]:
sns.boxplot(x = df['Present_Price'])

In [ ]:
df.dtypes

In [ ]:
list1=[]  #storing all the features having categorical datatype
for i in df.columns:
    if df[i].dtypes == "object":
         list1.append(i)

In [ ]:
cat_df=df[list1]
cat_df.head()

In [ ]:
df.head()

In [ ]:
df.describe().astype('int64')

**Data Visualization**

In [ ]:
sns.set_style("darkgrid")
sns.FacetGrid(df,hue="vehicle_age",height=10).map(plt.scatter,"Present_Price","selling_price").add_legend()
plt.show()

**More number of Years you will use your car lesser the amount you will get.**

In [ ]:
sns.set_style("darkgrid")
sns.FacetGrid(df,hue="Present_Price",height=10).map(plt.scatter,"km_driven","selling_price")
plt.xlabel("Present Price",fontsize=20)
plt.ylabel("Selling Price",fontsize=20)
plt.show()

**lesser the car would be driven higher will be the cost as we see the graph at max distance i e:- 3800000 kilometres the car’s cost is near to Zero or we can say nobody is willing to pay any amount to those cars.**

**plotting pair plot**

We cannot visualize multi dimensional scatter plot hence by using pair plot we can visualize each and every dimension of (Dimension with numerical variable )multidimensional data precisely.

In [ ]:
sns.pairplot(df, vars = ["vehicle_age", "km_driven", "seats", "selling_price", "Present_Price"])

As we see there are very less overlapping in dataset is seen so we cannot use knn ,linear regression,svm and because of the dynamic nature of dataset we even cannot use decision tree so we will go with random forest and xgboost.

**Uni variate analysis**

In [ ]:
sns.set_style("darkgrid")
sns.FacetGrid(df,height=10).map(sns.histplot,"selling_price")
plt.ylabel('No. of car sold')
plt.show()

In [ ]:
sns.set_style("darkgrid")
sns.FacetGrid(df,height=10).map(sns.histplot,"km_driven")
plt.xlabel("Distance Travel",Fontsize=20)
plt.ylabel("Demand",Fontsize=20)
plt.show()

Demand for those car that has been traveled less distance are in more demand especially if car has traveled distance within a range of 0–5000 kilometre people are more attracted towards them.

**multivariate Analysis**

In [ ]:
sns.set_style("darkgrid")
sns.jointplot("Present_Price","selling_price",data=df,kind="kde")
plt.show()

In [ ]:
list1

In [ ]:
df=df.drop(list1,axis=1)

In [ ]:
df.head()

Now we will do feature engineering on cat_df to convert the categorical variable into numerical variable.But before that we will check how many unique categorical variable each feature consists.

In [ ]:
cat_df.head()

In [ ]:
dict1={}
for index,col in enumerate(cat_df.columns):
    dict1[col]=cat_df[col].unique().tolist()
dict1

In [ ]:
cat_df=pd.get_dummies(cat_df,drop_first=True)
cat_df.head()

Now we will concatenate encoded cat_df with original df and will also delete the categorical variable column.

In [ ]:
df=pd.concat([df,cat_df],axis=1)
df.head()

In [ ]:
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12);

**Pre Modeling Steps**


Splitting dataset into dependent and independent variable.

In [ ]:
df.head()

In [ ]:
Y=df['selling_price']

In [ ]:
Y.head()

In [ ]:
X = df
X.drop(['selling_price'], axis = 1,inplace = True)

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=1)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

**Choosing best fit model for our dataset :**

checking which model will be best for our dataset as from pair plot it is clear that we have to take model which do prediction on non linear and combination of categorical and numerical data that are decision tree,random forest and xgboost then we will check which model will have high accuracy based on that we select the model. We can choose our best fit model using cross validation score.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn import model_selection
models=[]
models.append(('CART', DecisionTreeRegressor()))
models.append(("KNN",KNeighborsRegressor()))
models.append(("RF", RandomForestRegressor()))
models.append(("XGBOOST", XGBRegressor()))
names=[]
result=[]
for name,model in models:
    k_fold=model_selection.KFold(n_splits=10,shuffle=True,random_state=7)
    score=model_selection.cross_val_score(model,X_train,Y_train,cv=k_fold,scoring="r2")
    result.append(score)
    names.append(name)
    print(name,score.mean(),score.std())

In [ ]:
fig = plt.figure(figsize=(10,6))
plt.boxplot(result,labels=names)
plt.title('Algorithm Comparison',fontsize=25)
plt.show()

As we see above accuracy score result we can say that XGboost gives better accuracy with very low standard deviation hence we should go with XGboost.

In [ ]:
plt.figure(figsize=(8,6))
model=XGBRegressor()
model.fit(X,Y)
importance=np.sort(model.feature_importances_)
plt.barh(X.columns,importance)
plt.show()

**Hyper parameter tuning :**

In [ ]:
param_grid={"n_estimators":[100,120,130,140,150],
            "max_depth":range(1,10),
            "booster":["gbtree","gblinear","dart"]
           }
from sklearn.model_selection import RandomizedSearchCV
xgb=XGBRegressor()
random_cv=RandomizedSearchCV(estimator=xgb,param_distributions=param_grid,n_iter=10,cv=10)
random_cv.fit(X_train,Y_train)

In [ ]:
random_cv.best_params_

In [ ]:
from sklearn.metrics import r2_score 
xgb=XGBRegressor(n_estimators= 120, max_depth=5,booster= 'gbtree')
xgb.fit(X_train,Y_train)
Y_train_predicted=xgb.predict(X_train)
Y_test_predicted=xgb.predict(X_test)
print("Train set accuracy: ",r2_score(Y_train,Y_train_predicted))
print("Test set accuracy : ",r2_score(Y_test,Y_test_predicted))

**Understanding predicted values;**

In [ ]:
Result=pd.DataFrame({"Actual":Y_test,"Predicted":Y_test_predicted.astype('int64') })
Result.head(10)

In [ ]:
sns.kdeplot(Y_test,Y_test_predicted, color='b', shade=True)